In [1]:
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader
from monai.losses import DiceCELoss
from torch.optim import Adam
from dataset import VSDataset
from model import DynUNet

c:\Users\Acer\anaconda3\envs\3dVS\lib\site-packages\albumentations\__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.6' (you have '2.0.5'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
batch_size = 1
num_workers = 0
pin_memory = False
LEARNING_RATE = 1e-4
num_epochs = 4
image_size = 128

In [3]:
slice_transform = A.Compose([
    A.Resize(image_size, image_size),
    A.Rotate(limit=35, p=1.0),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.1),
    A.Normalize(mean=0.0, std=1.0, max_pixel_value=255.0),
    ToTensorV2()
])

In [ ]:
# masks_path = 'D:\\3dVS1\\sample_data\\Masks'
# images_path = 'D:\\3dVS1\\sample_data\\Image'
data_dir= r'D:\VSdata'
# csv_path = r'D:\VSdata\vs_paths_cleaned.csv' 
csv_path= r"C:\Users\Acer\Desktop\vs_paths.csv"

In [5]:
dataset = VSDataset(
    csv_path= csv_path,
    data_dir=data_dir,
    transform=slice_transform,  
    target_slices=128            
)

In [6]:
train_loader = DataLoader(
        dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=True,
    )

In [7]:
model= DynUNet(spatial_dims=3,
    in_channels=1,
    out_channels=1,
    kernel_size=[3, 3, 3, 3, 3, 3],
    strides=[1, 2, 2, 2, 2, 2],
    upsample_kernel_size=[2, 2, 2, 2, 2],
    res_block=True,
)

In [8]:
loss_function = DiceCELoss(to_onehot_y=True, softmax=True)
optimizer = Adam(model.parameters(), lr=1e-4)

In [9]:
device= torch.device('cpu')

In [10]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs = inputs.to(device)
        targets = targets.to(device)
        
        # optimizer.zero_grad()

        # outputs = model(inputs)
        # outputs = torch.sigmoid(outputs)
        # loss = loss_function(outputs, targets)
        # loss.backward()
        # optimizer.step()

        # running_loss += loss.item()
        
        # if batch_idx % 10 == 0:
        #     print(f"Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item()}")

    # avg_loss = running_loss / len(train_loader)
    # print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {avg_loss}")

print("Training complete!")

Reading image: D:\VSdata\Vestibular-Schwannoma-MC-RC\VS-MC-RC-030\05-11-1995-NA-t2of3  MRI IAMS-56472\7.000000-T2FIESTA-C-74895
Reading mask: D:\VSdata\VS-MC-RC segmentations-NIfTI-Release 2023/VS-MC-RC-030/1995-05-11/seg_T2.nii.gz
[INFO] image_stack shape already matches target shape.
[INFO] Final image_stack shape: (64, 512, 512)
(64, 512, 512) (64, 512, 512)
Reading image: D:\VSdata\Vestibular-Schwannoma-MC-RC\VS-MC-RC-079\04-05-1990-NA-t1of2  External Images for PACS-91613\2.000000-T2Ax 3D FIESTA-35837
Reading mask: D:\VSdata\VS-MC-RC segmentations-NIfTI-Release 2023/VS-MC-RC-079/1990-04-05/seg_T2.nii.gz
[INFO] image_stack shape already matches target shape.
[INFO] Final image_stack shape: (144, 512, 512)
(144, 512, 512) (144, 512, 512)
Reading image: D:\VSdata\Vestibular-Schwannoma-MC-RC\VS-SEG-042\07-16-1994-NA-t1of6 External Images for PACS-75965\401.000000-T2T23DDRIVE-71044
Reading mask: D:\VSdata\VS-MC-RC segmentations-NIfTI-Release 2023/VS-SEG-042/1994-07-16/seg_T2.nii.gz
[IN

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:\\VSdata\\Vestibular-Schwannoma-MC-RC\\VS-SEG-041\\11-30-1999\\image.nii.gz'

In [ ]:
csv_path = r'D:\VSdata\vs_paths.csv' 
import pandas as pd
df= pd.read_csv(csv_path)
df[df['patient_id']=='VS-MC-RC-036']

,patient_id,series_instance_uid,study_date,SegmentationPath,image_path,imagenifti_path
182,VS-MC-RC-036,1.3.6.1.4.1.14519.5.2.1.2356822478735907152433...,07-21-1995,VS-MC-RC segmentations-NIfTI-Release 2023/VS-M...,manifest-1742405880893\Vestibular-Schwannoma-M...,image-nifti\VS-MC-RC-036\07-21-1995\image.nii.gz
183,VS-MC-RC-036,1.3.6.1.4.1.14519.5.2.1.1160169313945234564236...,07-09-1999,VS-MC-RC segmentations-NIfTI-Release 2023/VS-M...,manifest-1742405880893\Vestibular-Schwannoma-M...,image-nifti\VS-MC-RC-036\07-09-1999\image.nii.gz


In [ ]:
from utils.utils import load_nifti_as_dhw
from utils.utils import dicom_load

image= r'D:\VSdata\image-nifti\VS-MC-RC-028\10-12-1992\image.nii.gz'
mask = r'D:\VSdata\VS-MC-RC segmentations-NIfTI-Release 2023/VS-MC-RC-028/1992-10-12/seg_T2.nii.gz'
path= r'D:\VSdata\Vestibular-Schwannoma-MC-RC\VS-MC-RC-028\10-12-1992-NA-t1of7  External Images for PACS-81342\3.000000-T2t2 tra 3d-ciss-71390'

mask= load_nifti_as_dhw(mask)
image= dicom_load(path, mask.shape)

print(image.shape)
print(mask.shape)

[INFO] Transposing image_stack from (40, 512, 384) to (40, 384, 512) using order (0, 2, 1)
[INFO] Final image_stack shape: (40, 384, 512)
(40, 384, 512)
(40, 384, 512)


In [ ]:
image= r'D:\VSdata\image-nifti\VS-MC-RC-028\10-12-1992\image.nii.gz'
image= load_nifti_as_dhw(image)
print(image.shape)
print(type(image))

(40, 384, 512)
<class 'numpy.ndarray'>


In [ ]:
import pandas as pd

csv_path = r'D:\VSdata\vs_paths.csv'
df = pd.read_csv(csv_path)

# Remove 'manifest-<digits>\\' from the beginning of the image_path
df['image_path'] = df['image_path'].str.replace(r'^manifest-\d+\\', '', regex=True)

# Save the cleaned dataframe back to CSV (overwrite original)
df.to_csv(r'D:\VSdata\vs_paths_cleaned.csv', index=False)
